# Workflow Summary

1. Load the application letters and the job ad
2. Embed the application letters and the job ad using a text embedding model
3. Find the most relevant (similar) application letters to the job ad
4. Create a custom prompt for the langauge model which includes the job ad and the most relevant application letters, plus some extra information
5. Generate a draft application letter by sending the prompt to the model and save it to a file for further use

# Importing Libraries

In [1]:
import io
import os
import tiktoken

import pandas as pd

from easy_letters import OpenAIConnector, Ranker
from easy_letters import LanguageModels, EmbeddingModels

from IPython.display import display, Markdown
from pathlib import Path

# Constants and Global Settings

In [2]:
pd.set_option('display.float_format', lambda x: f'{x:.2f}')

In [3]:
DATA_DIR = Path("../tests/test_data")
LETTERS_DIR = DATA_DIR/ "sample_letters"
SAMPLE_JOB_AD = DATA_DIR / "sample_ads/description_6.text"

OUTPUT_DIR = Path("./output")
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

LLM = LanguageModels.OPENAI_GPT4OMINI

# Helper Functions

In [4]:
class DocumentLoader:
    """A class to load documents from files."""
    
    @staticmethod
    def _read_txt(path: Path) -> str:
        with io.open(path, 'r', encoding='utf-8') as f:
            return f.read()

    def bulk_load_documents(self, path: Path, ext='.txt') -> pd.DataFrame:
        """Load all the documents in a directory with a specific extension into a DataFrame."""
        documents = []
        documents_ids = []
        
        ext = '.' + ext.lower().lstrip('.')
        for file in path.glob(f'*{ext}'):
            if ext in ('.txt', '.text'):
                documents.append(self._read_txt(file))
            else:
                raise ValueError(f'Unsupported file format: {ext.strip(".").capitalize()}')
            documents_ids.append(file.stem)
        return pd.DataFrame({'id': documents_ids, 'text': documents})

    def load_document(self, path: Path, ext='.txt') -> str:
        """Load a single document from a file."""
        if ext == '.txt':
            return self._read_txt(path)
        else:
            raise ValueError(f'Unsupported file format: {ext}')

In [5]:
def pprint(text: str):
    """Pretty print the text as markdown in Jupyter Notebook's output."""
    display(Markdown(text))

In [6]:
def gen_num_tokens(text: str, encoding: str = "r50k_base") -> int:
    """Calculate the number of tokens in a string for a specific model.
    Supported encodings and their compatible models:
    | Encoding    | Compatible Models                                     |
    |-------------|-------------------------------------------------------|
    | cl100k_base | gpt-4, gpt-3.5-turbo, and text-embedding-ada-002      |
    | p50k_base   | codex models, text-davinci-002, and text-davinci-003  |
    | r50k_base   | GPT-3-based models like davinci and GPT-3.5-turbo     |
    """
    encoding = tiktoken.get_encoding(encoding)
    return len(encoding.encode(text))

# Loading the Application Letters

In [7]:
doc_loader = DocumentLoader()
application_letters_df = doc_loader.bulk_load_documents(LETTERS_DIR, ext="text")

In [8]:
application_letters_df.head()

,id,text
0,sample_letter_708,Having worked in the tech industry for 7 years...
1,sample_letter_311,"Dear Hiring Manager,\n\nI am writing to expres..."
2,sample_letter_253,I am writing to apply for the Generative AI En...
3,sample_letter_611,"Dear Hiring Manager,\n\nI am writing to expres..."
4,sample_letter_394,"Dear Hiring Manager,\n\nI am writing to expres..."


# Initializing a Connector to the OpenAI API

In [9]:
openai_key = os.getenv("OPENAI_API_KEY")

# Check if the API key is set and available
assert openai_key != "", "Please set the OPENAI_API_KEY environment variable"

In [10]:
connector = OpenAIConnector(openai_key)

# Embedding the Letters

In [11]:
letters_with_embeddings_df = application_letters_df.copy()
letters_with_embeddings_df['embedding'] = connector.embed(documents=application_letters_df['text'], 
                                                          model=EmbeddingModels.OPENAPI_EMS)

In [12]:
letters_with_embeddings_df.head()

,id,text,embedding
0,sample_letter_708,Having worked in the tech industry for 7 years...,"[-0.015994248911738396, 0.020503530278801918, ..."
1,sample_letter_311,"Dear Hiring Manager,\n\nI am writing to expres...","[0.035636693239212036, -0.001379814581014216, ..."
2,sample_letter_253,I am writing to apply for the Generative AI En...,"[0.017323147505521774, -0.044040385633707047, ..."
3,sample_letter_611,"Dear Hiring Manager,\n\nI am writing to expres...","[0.049760978668928146, -0.015271708369255066, ..."
4,sample_letter_394,"Dear Hiring Manager,\n\nI am writing to expres...","[0.034355904906988144, 0.02534237876534462, 0...."


# Finding the Most Relevant Letters to the Job Ad

In [13]:
ranker = Ranker()

In [14]:
ranker.make_collection(documents_with_embeddings=letters_with_embeddings_df)

Creating collection letters with 813 points of size 1536


In [15]:
sample_job_ad = doc_loader.load_document(path=SAMPLE_JOB_AD)

pprint(sample_job_ad)

OSI Assistance Foundation - Armenian Branch Office
JOB TITLE:  Chief Accountant/ Finance Assistant
POSITION LOCATION: Yerevan, Armenia
JOB DESCRIPTION:   The Armenian Branch Office of the Open Society
Institute Assistance Foundation is seeking applications for the position
of Chief Accountant/ Finance Assistant. The Chief Accountant/ Finance
Assistant will be responsible for all transactions, connected with grant
payments, administrative expenses. 
REQUIRED QUALIFICATIONS: 
- University degree in finance/ accounting; 
- One year minimum experience in an international organization; 
- Strong organizational skills; 
- Good knowledge of software programs: MS Excel and MS Access; 
- Good knowledge of IAS, Armenian taxation laws, reporting requirements
and current reforms; 
- Discretion and ability to handle confidential issues; 
- Self-motivation with an ability to set and meet goals; 
- Quick learning skills; 
- Fluency in English, Armenian and Russian. 
APPLICATION PROCEDURES:  For submission of applications/ CVs, please
contact the OSI AF - Armenia at: 1 Pushkin Str., apt. 2. Tel: 54 2119,
54 39 01, 54 17 19; e-mail: jobs@.... 
Please clearly mention in your application letter that you learned of
this job opportunity through Career Center and mention the URL of its
website - www.careercenter.am, Thanks.
APPLICATION DEADLINE:   16 January 2004, 6:00 pm.
----------------------------------
To place a free posting for job or other career related opportunities in
your organization at careercenter.am website, e-mail us atmailbox@...

In [16]:
sample_job_ad_embedded = connector.embed(documents=[sample_job_ad], 
                                         model=EmbeddingModels.OPENAPI_EMS)

In [17]:
sample_job_ad_embedded

[array([-0.03499759, -0.01236026,  0.00740111, ...,  0.00064774,
         0.02555379, -0.00236529])]

In [18]:
most_relevant_letters =  ranker.find_similar(sample_job_ad_embedded[0], top_k=3, min_similarity=0.1)

most_relevant_letters

[ScoredPoint(id=478, version=0, score=0.36129672473127655, payload={'text': 'I am applying for the System Administrator position at Adobe. My extensive experience with Red Hat, CentOS, and Puppet, as well as my proficiency in Ruby, makes me a strong candidate for this role. I am excited about the opportunity to bring my unique skill set to your team.'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=644, version=0, score=0.35730961122204624, payload={'text': 'I am very interested in the Senior Support Engineer position at Innovative Tech. I have over 4 years of experience in IT support, and I am confident that I can bring a high level of expertise to your team. In my current role at ABC Inc, I have been responsible for managing a team of support specialists and dealing with complex technical issues. I am highly skilled in troubleshooting, networking, and both Linux and Windows systems. I hold a BSc in Information Technology and I am always eager to learn and improve my

In [19]:
for letter in most_relevant_letters:
    print("="*80)
    print(f"Letter ID: {letter.id}")
    print(f"Similarity: {letter.score:.2f}")
    print(f"Letter Text:\n{letter.payload['text']}")
    print()

Letter ID: 478
Similarity: 0.36
Letter Text:
I am applying for the System Administrator position at Adobe. My extensive experience with Red Hat, CentOS, and Puppet, as well as my proficiency in Ruby, makes me a strong candidate for this role. I am excited about the opportunity to bring my unique skill set to your team.

Letter ID: 644
Similarity: 0.36
Letter Text:
I am very interested in the Senior Support Engineer position at Innovative Tech. I have over 4 years of experience in IT support, and I am confident that I can bring a high level of expertise to your team. In my current role at ABC Inc, I have been responsible for managing a team of support specialists and dealing with complex technical issues. I am highly skilled in troubleshooting, networking, and both Linux and Windows systems. I hold a BSc in Information Technology and I am always eager to learn and improve my skills. I believe that my experience and dedication would make me a valuable addition to your company. Thank you 

# Gluing Everything Together

In [20]:
prompt_part_1 = "I'm applying for a job with this description:\n\n"
prompt_part_2 = "#START OF JOB AD\n\n" + sample_job_ad + "\n\n#END OF JOB AD"
prompt_part_3 = "\n\nI need to submit a application letter with my CV. Here is a few examples of my previous application letters:\n\n"
prompt_part_4 = "\n\n".join([('#START OF EXAMPLE APPLICATION LETTER\n\n'+
                              t.payload['text'] + '\n\n#END OF EXAMPLE APPLICATION LETTER')
                              for t in most_relevant_letters])
# Extra information for the prompt
prompt_part_5 = ("\n\nWrite a new application letter that is tailored to the job description above. "
                 "Be concise and to the point. The letter should be no longer than 500 words. "
                 "The letter should be written in English and be easy to read.\n\n")

prompt = prompt_part_1 + prompt_part_2 + prompt_part_3 + prompt_part_4 + prompt_part_5
pprint(prompt)

I'm applying for a job with this description:

#START OF JOB AD

OSI Assistance Foundation - Armenian Branch Office
JOB TITLE:  Chief Accountant/ Finance Assistant
POSITION LOCATION: Yerevan, Armenia
JOB DESCRIPTION:   The Armenian Branch Office of the Open Society
Institute Assistance Foundation is seeking applications for the position
of Chief Accountant/ Finance Assistant. The Chief Accountant/ Finance
Assistant will be responsible for all transactions, connected with grant
payments, administrative expenses. 
REQUIRED QUALIFICATIONS: 
- University degree in finance/ accounting; 
- One year minimum experience in an international organization; 
- Strong organizational skills; 
- Good knowledge of software programs: MS Excel and MS Access; 
- Good knowledge of IAS, Armenian taxation laws, reporting requirements
and current reforms; 
- Discretion and ability to handle confidential issues; 
- Self-motivation with an ability to set and meet goals; 
- Quick learning skills; 
- Fluency in English, Armenian and Russian. 
APPLICATION PROCEDURES:  For submission of applications/ CVs, please
contact the OSI AF - Armenia at: 1 Pushkin Str., apt. 2. Tel: 54 2119,
54 39 01, 54 17 19; e-mail: jobs@.... 
Please clearly mention in your application letter that you learned of
this job opportunity through Career Center and mention the URL of its
website - www.careercenter.am, Thanks.
APPLICATION DEADLINE:   16 January 2004, 6:00 pm.
----------------------------------
To place a free posting for job or other career related opportunities in
your organization at careercenter.am website, e-mail us atmailbox@...

#END OF JOB AD

I need to submit a application letter with my CV. Here is a few examples of my previous application letters:

#START OF EXAMPLE APPLICATION LETTER

I am applying for the System Administrator position at Adobe. My extensive experience with Red Hat, CentOS, and Puppet, as well as my proficiency in Ruby, makes me a strong candidate for this role. I am excited about the opportunity to bring my unique skill set to your team.

#END OF EXAMPLE APPLICATION LETTER

#START OF EXAMPLE APPLICATION LETTER

I am very interested in the Senior Support Engineer position at Innovative Tech. I have over 4 years of experience in IT support, and I am confident that I can bring a high level of expertise to your team. In my current role at ABC Inc, I have been responsible for managing a team of support specialists and dealing with complex technical issues. I am highly skilled in troubleshooting, networking, and both Linux and Windows systems. I hold a BSc in Information Technology and I am always eager to learn and improve my skills. I believe that my experience and dedication would make me a valuable addition to your company. Thank you for considering my application.

#END OF EXAMPLE APPLICATION LETTER

#START OF EXAMPLE APPLICATION LETTER

Dear Hiring Manager,

I am writing to express my interest in the Intermediate Applied Scientist position at ABC Environmental Solutions. With my strong background in environmental science and experience in applied consulting, I believe I am well-suited for this role.

In my current position as an Applied Scientist at DEF Environmental Solutions, I have successfully conducted environmental impact assessments and provided regulatory support. I have also gained proficiency in data analysis using tools such as Microsoft Office Suite and Arc GIS. Additionally, my fieldwork experience has allowed me to develop strong problem-solving skills and the ability to thrive under new challenges.

I hold a Bachelor's degree in Environmental Science and have completed the Canadian Certified Electrofishing Course, demonstrating my commitment to continuous learning and professional development. Furthermore, my Level 1 First Aid Certificate in BC showcases my dedication to safety in the field.

I am confident that my technical expertise, leadership abilities, and strong communication skills make me a valuable asset to your team. I am excited about the opportunity to contribute to ABC Environmental Solutions and make a positive impact on environmental conservation.

Thank you for considering my application. I look forward to the opportunity to discuss how my qualifications align with the requirements of the Intermediate Applied Scientist position.

Sincerely,
John Smith

#END OF EXAMPLE APPLICATION LETTER

Write a new application letter that is tailored to the job description above. Be concise and to the point. The letter should be no longer than 500 words. The letter should be written in English and be easy to read.



In [21]:
num_tokens = gen_num_tokens(text=prompt, encoding='r50k_base')
print(f"Number of tokens sent to the API: {num_tokens}")

Number of tokens sent to the API: 995


In [22]:
draft_letter = connector.chat(prompt=prompt, 
                          model=LLM,
                          temperature=0.1,
                          max_tokens=512)

In [23]:
pprint(draft_letter)

[Your Name]  
[Your Address]  
[City, Zip Code]  
[Your Email]  
[Your Phone Number]  
[Date]  

Hiring Manager  
OSI Assistance Foundation - Armenian Branch Office  
1 Pushkin Str., apt. 2  
Yerevan, Armenia  

Dear Hiring Manager,

I am writing to express my interest in the Chief Accountant/Finance Assistant position at the OSI Assistance Foundation, as advertised. With a university degree in finance and over a year of experience working in an international organization, I am confident in my ability to contribute effectively to your team.

In my previous role at [Your Previous Company], I was responsible for managing financial transactions, including grant payments and administrative expenses. This experience has equipped me with a solid understanding of International Accounting Standards (IAS) and Armenian taxation laws, as well as the reporting requirements necessary for compliance. I have developed strong organizational skills that enable me to handle multiple tasks efficiently while maintaining attention to detail.

I am proficient in MS Excel and MS Access, which I have used extensively for data analysis and financial reporting. My ability to quickly learn new software and adapt to changing environments has been a key factor in my success. I pride myself on my discretion and ability to handle confidential information with the utmost professionalism.

Fluency in English, Armenian, and Russian allows me to communicate effectively with diverse stakeholders, enhancing collaboration and understanding within the team. I am self-motivated and committed to setting and achieving goals, which I believe aligns well with the values of the OSI Assistance Foundation.

I am excited about the opportunity to bring my skills and experience to your organization and contribute to the important work you do. Thank you for considering my application. I look forward to the possibility of discussing how I can support the financial operations of the OSI Assistance Foundation.

Sincerely,

[Your Name]

In [24]:
# Save the draft letter to a file
output_file = OUTPUT_DIR / f"draft-letter-using-{LLM}.txt"
with io.open(output_file, 'w', encoding='utf-8') as f:
    f.write(draft_letter)

In [25]:
print("Done! The draft letter is saved to the file: ", output_file)

Done! The draft letter is saved to the file:  output/draft-letter-using-gpt-4o-mini.txt
